In [ ]:
# install required libraries
! pip install rasterio
! pip install
!apt-get update
!apt-get install gdal-bin

ERROR: You must give at least one requirement to install (see "pip help install")
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry

In [ ]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from google.colab import drive
import rasterio
from rasterio.features import shapes
from rasterio.merge import merge
import os
import glob
import subprocess
import dask.dataframe as dd
from dask import delayed
from pyproj import Transformer


In [ ]:
# Mount drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load the dataset - Change the name of the input file
terrain_path = '/content/drive/MyDrive/0.Data science final project/4. Terrain data/Outputs/chunk_r1_c4.tif' #<-----------------------------------------CHANGE


In [ ]:
# Find the coordinate bounds for the loaded dataset
with rasterio.open(terrain_path) as src:
    bounds = src.bounds
    print(bounds)

BoundingBox(left=-116.20013888888045, bottom=42.20013888888653, right=-114.00013888888016, top=45.60013888888698)


In [ ]:
# Reproject the file to UTM Zone 10 or 11 based on the longitude value, if in the middle pick the zone where most of the data sits (below 120W its zone10, above 120W its zone11)
input_tiff = terrain_path

# Change the name of the output file
projected_output = r'/content/drive/MyDrive/0.Data science final project/4. Terrain data/Projected outputs/projected_chunk_r1_c4.tif' #<--------- CHANGE

# Reproject to UTM 10N/11N
print("Reprojecting to UTM zone 11N...") #<----------------------------------------------------------------------------------------------------CHANGE
result = subprocess.run([
    "gdalwarp",
    "-t_srs",
    "EPSG:32611", #use 32610 or 32611 #<-------------------------------------------------------------------------------------------------------CHANGE
    input_tiff,
    projected_output
], capture_output=True, text=True)
print(f"gdalwarp stderr: {result.stderr}")

print(f"Data reprojected and saved to {projected_output}")


Reprojecting to UTM zone 11N...
gdalwarp stderr: 
Data reprojected and saved to /content/drive/MyDrive/0.Data science final project/4. Terrain data/Projected outputs/projected_chunk_r1_c4.tif


In [ ]:
# Ran the code without Dask and without chunking... estimate dcompletion for both of those was over 2.5hours
# With chunking, able to process in a few mins

# Extract elevation, slope and aspect from the projected output

def extract_terrain_data(input_tiff_path, output_csv_path, chunk_size=1024):  # Added chunk_size

    slope_tiff = "slope.tif"
    aspect_tiff = "aspect.tif"

    print(f"Calculating slope for {input_tiff_path}...")
    try:
        subprocess.run(["gdaldem", "slope", input_tiff_path, slope_tiff], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error calculating slope: {e}")
        return

    print(f"Calculating aspect for {input_tiff_path}...")
    try:
        subprocess.run(["gdaldem", "aspect", input_tiff_path, aspect_tiff], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error calculating aspect: {e}")
        return

    with rasterio.open(input_tiff_path) as src_elevation, \
         rasterio.open(slope_tiff) as src_slope, \
         rasterio.open(aspect_tiff) as src_aspect:

        transform = src_elevation.transform
        height = src_elevation.height
        width = src_elevation.width
        total_pixels = height * width
        processed_pixels = 0

        # Initialize lists to store data
        x_coords = []
        y_coords = []
        elevations = []
        slopes = []
        aspects = []

        # Iterate through chunks
        for row_offset in range(0, height, chunk_size):
            for col_offset in range(0, width, chunk_size):
                chunk_width = min(chunk_size, width - col_offset)
                chunk_height = min(chunk_size, height - row_offset)

                window = rasterio.windows.Window(col_offset, row_offset, chunk_width, chunk_height)

                elevation_chunk = src_elevation.read(1, window=window)
                slope_chunk = src_slope.read(1, window=window)
                aspect_chunk = src_aspect.read(1, window=window)

                for row in range(chunk_height):
                    for col in range(chunk_width):
                        x, y = transform * (col + col_offset, row + row_offset)

                        elevation = elevation_chunk[row, col]
                        slope = slope_chunk[row, col]
                        aspect = aspect_chunk[row, col]

                        elevation = np.nan if elevation == src_elevation.nodata else elevation
                        slope = np.nan if slope == src_slope.nodata else slope
                        aspect = np.nan if aspect == src_aspect.nodata else aspect

                        x_coords.append(x)
                        y_coords.append(y)
                        elevations.append(elevation)
                        slopes.append(slope)
                        aspects.append(aspect)

                        processed_pixels += 1
                        if processed_pixels % 1000000 == 0:
                            print(f"Processed {processed_pixels}/{total_pixels} pixels in {input_tiff_path}")

        # Create Pandas DataFrame
        print(f"Creating Pandas DataFrame for {input_tiff_path}...")
        df = pd.DataFrame({
            'x': x_coords,
            'y': y_coords,
            'elevation': elevations,
            'slope': slopes,
            'aspect': aspects
        })

        # Save to CSV
        print(f"Saving to CSV for {input_tiff_path}...")
        df.to_csv(output_csv_path, index=False)

    os.remove(slope_tiff)
    os.remove(aspect_tiff)

# Run the function
# Chaange the input and output files names
input_tiff = r'/content/drive/MyDrive/0.Data science final project/4. Terrain data/Projected outputs/projected_chunk_r1_c4.tif' #<--------------CHANGE
output_csv = r'/content/drive/MyDrive/0.Data science final project/4. Terrain data/Extracted Data/chunk_r1_c4_terrain_data.csv' #<--------------CHANGE

extract_terrain_data(input_tiff, output_csv)

print(f"Terrain data extracted and saved to {output_csv}")

Calculating slope for /content/drive/MyDrive/0.Data science final project/4. Terrain data/Projected outputs/projected_chunk_r1_c4.tif...
Calculating aspect for /content/drive/MyDrive/0.Data science final project/4. Terrain data/Projected outputs/projected_chunk_r1_c4.tif...
Processed 1000000/86442432 pixels in /content/drive/MyDrive/0.Data science final project/4. Terrain data/Projected outputs/projected_chunk_r1_c4.tif
Processed 2000000/86442432 pixels in /content/drive/MyDrive/0.Data science final project/4. Terrain data/Projected outputs/projected_chunk_r1_c4.tif
Processed 3000000/86442432 pixels in /content/drive/MyDrive/0.Data science final project/4. Terrain data/Projected outputs/projected_chunk_r1_c4.tif
Processed 4000000/86442432 pixels in /content/drive/MyDrive/0.Data science final project/4. Terrain data/Projected outputs/projected_chunk_r1_c4.tif
Processed 5000000/86442432 pixels in /content/drive/MyDrive/0.Data science final project/4. Terrain data/Projected outputs/projec

In [ ]:
# Since the csv created has x and y in metres, they need to be converted back into lat/lon degrees

def convert_utm_to_latlon(csv_file, output_csv):
    """
    Converts x and y coordinates from UTM to latitude/longitude.

    Args:
        csv_file: Path to the input CSV file with UTM coordinates.
        output_csv: Path to the output CSV file with latitude/longitude coordinates.
    """

    df = pd.read_csv(csv_file)

    # Define the CRS transformation
    utm_crs = 'EPSG:32611'  # UTM Zone 10N  # <--------------------------------------------------------------------------------------------------------CHANGE
    latlon_crs = 'EPSG:4326'  # WGS 84 (latitude/longitude)
    transformer = Transformer.from_crs(utm_crs, latlon_crs, always_xy=True) #always xy ensures that the order of the coordinates is x,y

    # Convert x and y coordinates to latitude and longitude
    lons, lats = transformer.transform(df['x'].values, df['y'].values)

    # Replace x and y with latitude and longitude
    df['x'] = lons
    df['y'] = lats

    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_csv, index=False)

# Example usage:
input_csv = r'/content/drive/MyDrive/0.Data science final project/4. Terrain data/Extracted Data/chunk_r1_c4_terrain_data.csv'  # <---------------------CHANGE
output_csv = r'/content/drive/MyDrive/0.Data science final project/4. Terrain data/Latlon data/chunk_r1_c4_latlon.csv'       # <---------------------CHANGE

convert_utm_to_latlon(input_csv, output_csv)

print(f"UTM coordinates converted to latitude/longitude and saved to {output_csv}")

UTM coordinates converted to latitude/longitude and saved to /content/drive/MyDrive/0.Data science final project/4. Terrain data/Latlon data/chunk_r1_c4_latlon.csv
